In [ ]:
%pip install --user conversation_analytics_toolkit


In [ ]:
pip install ibm-cloud-sdk-core

In [ ]:
import nltk
nltk.download('words')
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:


import json
import pandas as pd
from pandas.io.json import json_normalize
from IPython.core.display import display, HTML

from pandas.io.json import json_normalize
from IPython.core.display import display, HTML

import pandas as pd
import argparse



from urllib.parse import urlparse, parse_qs
from datetime import datetime

c_RESPONSE = 'response'
c_CONTEXT = 'context'
c_SYSTEM = 'system'
c_INPUT = 'input'
c_OUTPUT = 'output'
c_INTENTS = 'intents'
c_INTENT = 'intent'
c_TEXT = 'text'
c_BRANCH_EXITED_REASON = 'branch_exited_reason'
c_LOG_MESSAGING = 'log_messaging'
c_CONFIDENCE = 'confidence'
c_LOGS = 'logs'
c_PAGINATION = 'pagination'
c_NEXT_URL = 'next_url'
c_CURSOR = 'cursor'

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 20000)
pd.set_option('display.max_colwidth', 10000000)

In [ ]:
import conversation_analytics_toolkit
from conversation_analytics_toolkit import wa_assistant_skills
from conversation_analytics_toolkit import transformation
from conversation_analytics_toolkit import filtering2 as filtering
from conversation_analytics_toolkit import analysis 
from conversation_analytics_toolkit import visualization 
from conversation_analytics_toolkit import selection as vis_selection
from conversation_analytics_toolkit import wa_adaptor 
from conversation_analytics_toolkit import transcript 
from conversation_analytics_toolkit import flows 
from conversation_analytics_toolkit import keyword_analysis 
from conversation_analytics_toolkit import sentiment_analysis 
import ibm_watson
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator,BasicAuthenticator
from ibm_watson import AssistantV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson import AssistantV1 as WatsonAssistant

authenticator = IAMAuthenticator("COLOQUE A APIKEY") 
service = AssistantV1(version='2021-11-27',authenticator = authenticator) #COLOQUE A VERSÂO
service.set_service_url("https://api.us-south.assistant.watson.cloud.ibm.com") #COLOQUE A URL





In [ ]:
#MOSTRA A WORKSPACE_ID DE CADA WORKSPACE (SEJA PROD OU HOMOLOG)
response=service.list_workspaces().get_result() 
print(json.dumps(response, indent=2))

In [ ]:
#AQUI ELE FAZ A BUSCA DAS LOGS

workspace_id = 'COLOQUE A WORKSPACE_ID'
workspace=service.get_workspace(
    workspace_id=workspace_id,
    export=True
).get_result()

limit_number_of_records=100000000
query_filter = "response_timestamp>=2022-06-07,response_timestamp<2022-06-09" #COLOQUE A DATA
df_logs = wa_adaptor.read_logs(service, workspace_id, limit_number_of_records, query_filter)

In [ ]:
skill_id = workspace_id
assistant_skills = wa_assistant_skills.WA_Assistant_Skills()
assistant_skills.add_skill(skill_id, workspace)
#VALIDA A QUANTIDADE DE WORKSPACE ID
print("workspace_ids nas skills: " + pd.DataFrame(assistant_skills.list_skills())["skill_id"].unique())
print("workspace_ids nas logs: "+ df_logs.workspace_id.unique())

In [ ]:
df_logs_canonical = transformation.to_canonical_WA_v2(df_logs) #TRANSFORMA AS CONVERSAS EM LOGS
#df_logs_canonical = transformation.to_canonical_WA_v2(df_logs, assistant_skills, skill_id_field="workspace_id", include_nodes_visited_str_types=True, include_context=False)

In [ ]:
df_logs_to_analyze = df_logs_canonical.copy(deep=False) #IDENTIFICA LOGS CANONICAS

In [ ]:
#AQUI ELE FAZ A BUSCA DAS CONVERSAS PARA GERAR O EXCEL

j = []
page_count = 1
cursor = None
count = 0
while True:
   
    print('Reading page {}.'.format(page_count))
    x = service.list_logs(workspace_id=workspace_id,filter = query_filter,cursor=cursor,page_limit=500).result
    j.append(x[c_LOGS])
    count = count + len(x[c_LOGS])
    page_count = page_count + 1
    cursor= x.get('pagination').get('next_cursor')
    if not cursor:
        break
        

In [ ]:
#GERA O EXCEL

i = 0 
c = 0
df = pd.DataFrame(columns=['Código da conversa','Conversa','Resposta','Data'])
for i in range(len(j)):
    m = 0
    for m in range(len(j[i])):
                                
        if 'conversation_id' in j[i][m]['response']['context']:
                df.loc[c,'Código da conversa'] = j[i][m]['response']['context']['conversation_id']
                df.loc[c,'Conversa'] = j[i][m]['response']['input']['text']
                df.loc[c,'Resposta'] = j[i][m]['response']['output']['generic']
                df.loc[c,'Data'] = j[i][m]['request_timestamp']
        c = c +1

In [ ]:
#EXPORTA O EXCEL
df.to_excel(r'C:\Users\conversas.xlsx', index = False)#indicar o caminho onde a pasta será salva

In [ ]:
#ESTE É PARA IDENTIFICAR AS $CALLSUBJECTS
df_logs.head(100)